# 🔐 ARNN per Rilevamento SYN Flood – H-DIR²
Training, valutazione e ROC Curve

In [ ]:
# 📦 Installazione librerie necessarie (solo su Colab)
!pip install scikit-learn torch matplotlib pandas


In [ ]:
# 📁 Upload del file V1
from google.colab import files
uploaded = files.upload()


In [ ]:
# 📊 Caricamento CSV vettoriale
import pandas as pd
df = pd.read_csv('O3_Vectorization_V1_matrix.csv')

# 🎯 Selezione feature e label
X = df[['delta_H', 'r', 'syn_count', 'duration_mean', 'syn_ratio']].values
y = df['label'].values


In [ ]:
# 🔀 Train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# 🧠 Definizione ARNN
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Tensori
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32)


In [ ]:
class ARNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ARNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.out = nn.LogSoftmax(dim=1)

    def forward(self, x):
        return self.out(self.fc2(self.relu(self.fc1(x))))


In [ ]:
# 🔧 Inizializzazione e training
model = ARNN(5, 32, 2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()

for epoch in range(30):
    model.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()


In [ ]:
# 🧪 Valutazione
model.eval()
y_true, y_pred, y_prob = [], [], []

with torch.no_grad():
    for xb, yb in test_loader:
        out = model(xb)
        probs = torch.exp(out)[:, 1]
        preds = torch.argmax(out, dim=1)
        y_true.extend(yb.numpy())
        y_pred.extend(preds.numpy())
        y_prob.extend(probs.numpy())

print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))
print("ROC AUC Score:", roc_auc_score(y_true, y_prob))


In [ ]:
# 📈 ROC Curve
fpr, tpr, _ = roc_curve(y_true, y_prob)
plt.plot(fpr, tpr, label='ARNN')
plt.plot([0, 1], [0, 1], '--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - ARNN SYN Flood Detection')
plt.legend()
plt.grid(True)
plt.show()
